In [13]:
# Saving the Best model...

import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import time

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.feature_selection import SequentialFeatureSelector

#from sklearn.feature_selection import SelectKBest
#from sklearn.feature_selection import chi2
#from sklearn.feature_selection import RFE


In [14]:
def forward_selection(indep, dep,n):
    logistic_model=LogisticRegression(solver='saga', max_iter=1000)
    
    fs = SequentialFeatureSelector(estimator=logistic_model, n_features_to_select=n, direction='forward', scoring=None, cv=5, n_jobs=1)
    
    fit1 = fs.fit(indep, dep)
    x_new= fit1.transform(indep)
    
    selected_features= indep.columns[fit1.get_support()]
    
    return x_new, selected_features


def Split_Scaler(indep,dep):
    x_train,x_test,y_train,y_test=train_test_split(indep,dep,test_size=0.30,random_state=0)
    scX=StandardScaler()
    x_train=scX.fit_transform(x_train)
    x_test=scX.transform(x_test)
    return x_train,x_test,y_train,y_test,scX

#    # save Standard scalar obj
#    pickle.dump(scX,open("scx.pkl",'wb'))

def cm_predition(classifier, x_test):
    y_pred=classifier.predict(x_test)

    #making the Confusion Matrix
    from sklearn.metrics import confusion_matrix
    cm=confusion_matrix(y_test,y_pred)

    # making Classification Report
    from sklearn.metrics import classification_report
    clf_report=classification_report(y_test,y_pred)

    # finding the accuracy score
    from sklearn.metrics import accuracy_score
    accuracy_score=accuracy_score(y_test,y_pred)
    
    return classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred

def logistic(x_train,y_train,x_test):
    from sklearn.linear_model import LogisticRegression
    classifier=LogisticRegression(random_state=0)
    classifier.fit(x_train,y_train)
    classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred = cm_predition(classifier,x_test)
    return classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred 
    

# Displaying the values in the tabular frame

def FE_selection(accuracy_score):
    
    df=pd.DataFrame(index=["Logistic"],columns=["Logistic"])
    df["Logistic"]=accuracy_score
    
    return df




In [15]:
dataset=pd.read_csv("prep.csv",index_col=None)
df1=dataset  

#Convert categorical variable into dummy/indicator variables.
df1=pd.get_dummies(df1,drop_first=True,dtype=int)
df1

# separation of independent and dependent variables
indep=df1.drop("classification_yes",axis=1)
dep=df1["classification_yes"]
  
    
x_new, selected_features =forward_selection (indep, dep,4)



/opt/anaconda3/envs/venvaiml/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/venvaiml/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/venvaiml/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/venvaiml/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/venvaiml/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/venvaiml/lib/py

In [16]:
x_train,x_test,y_train,y_test,scX = Split_Scaler(x_new, dep) 

classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred = logistic(x_train,y_train,x_test)

result= FE_selection(accuracy_score)

In [17]:
print("Accuracy: ",accuracy_score)

Accuracy:  0.9833333333333333


In [18]:
result #4

,Logistic
Logistic,0.983333


In [19]:
print ("seleceted features: ",selected_features)

seleceted features:  Index(['al', 'hrmo', 'sg_c', 'sg_d'], dtype='object')


In [20]:
# Save the model

pickle.dump(classifier,open("log_model","wb"))

# save Standard scalar obj
pickle.dump(scX,open("scx01.pkl",'wb'))

In [21]:
df1[selected_features]

,al,hrmo,sg_c,sg_d
0,3.0,12.518156,1,0
1,2.0,10.700000,1,0
2,1.0,12.000000,0,0
3,1.0,8.100000,0,1
4,0.0,11.800000,1,0
...,...,...,...,...
394,0.0,12.500000,0,0
395,0.0,8.700000,1,0
396,3.0,9.100000,1,0
397,0.0,8.500000,0,0
